In [ ]:
#%matplotlib inline

# This examples shows who to download files from the ONC server
import os

import numpy as np
import pandas
import scipy.ndimage

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib

import strawb.tools

# Load ONC DB and mask files of interest

In [ ]:
# Check if DB exits, if not load it, but update it anyway
if os.path.exists(strawb.Config.pandas_file_sync_db):
    db = strawb.SyncDBHandler()  # loads the db from disc
else:
    db = strawb.SyncDBHandler(load_db=False)  # loads the db from ONC server

db.load_onc_db_update(output=True, save_db=True)

In [ ]:
# mask by device and data-product
mask = db.dataframe['deviceCode'] == 'TUMMUONTRACKER001'
mask &= db.dataframe.dataProductCode == 'MTSD'  # see SyncDBHandler.sensor_mapping

In [ ]:
db.dataframe.dateTo.max()

In [ ]:
# show some masked entries
db.dataframe[mask].iloc[-5:]

# Download a specific file, needs some minutes if file isn't synced

In [ ]:
file_list = ['TUMMUONTRACKER001_20220731T001146.146Z-SDAQ-MUON.hdf5']
db_i = db.get_files_from_names(file_list)
db_i

# Load the file

In [ ]:
muon = strawb.MuonTracker(file_list[0])

## show file version - 4 has tot data, which we need

In [ ]:
muon.file_handler.file_version

In [ ]:
plt.figure()

plt.plot(muon.file_handler.tot_time_ns[:])
plt.xlabel('index')
plt.ylabel('time [s]')
plt.tight_layout()

## Show data structure

In [ ]:
# all have the same shape -> entry 'i' represents one event
print(f'tot_time   : {muon.file_handler.tot_time.shape}')
print(f'tot_time_ns: {muon.file_handler.tot_time_ns.shape}')
print(f'tot_channel: {muon.file_handler.tot_channel.shape}')
print(f'tot_tot    : {muon.file_handler.tot_tot.shape}')

In [ ]:
# show the first n events
n = 5

# Absolute Timestamp in seconds [s] with resolution [ns]
print(f'tot_time   : {muon.file_handler.tot_time[:n]}')

# Absolute Timestamp in seconds [s] with resolution [ns], converted to dateformat
print(f'tot_time   : {muon.file_handler.tot_time.asdatetime()[:n]}')

# TRB Timestamp in [ns] - not absolute therefore most precise
print(f'tot_time_ns: {muon.file_handler.tot_time_ns[:n]}')

# TRB Channel of the event
print(f'tot_channel: {muon.file_handler.tot_channel[:n]}')

# TRB time over threshold in [ns]
print(f'tot_tot    : {muon.file_handler.tot_tot[:n]}')

# Load and Cut Data
It seems, the TRB has an overflow at 2750 in both time data. (can be corrected with np.unwrap)

In [ ]:
n = 1000000
tot_time_ns = muon.file_handler.tot_time_ns[:n]

mask_valid = tot_time_ns!=1e-9

plt.figure()
plt.plot(tot_time_ns[mask_valid], lw=3, label='raw')
period=2750
plt.plot(np.unwrap(tot_time_ns[mask_valid], period=period), label=period)
period=2749
plt.plot(np.unwrap(tot_time_ns[mask_valid], period=period), label=period)

period=1000
plt.plot(np.unwrap(tot_time_ns[mask_valid], period=period), label=period)

period=2748.75
plt.plot(np.unwrap(tot_time_ns[mask_valid], period=period), label=period)

period=2**38*1e-8
plt.plot(np.unwrap(tot_time_ns[mask_valid], period=period), label=period)
period=2**39*1e-8
plt.plot(np.unwrap(tot_time_ns[mask_valid], period=period), label=period)
plt.legend()
# plt.xlim((842581.3723510662, 842594.7912079565))
# plt.ylim((7300, 7310))

In [ ]:
tot = muon.file_handler.tot_tot[:]

tot_time_ns = muon.file_handler.tot_time_ns[:]

# some values are 1ns, exclude them
mask_valid = tot_time_ns!=1e-9

# some tot values are very high ~1e12, exclude them here or not
max_tot = 1 * 1e3 # cut at 1us - [ns]
mask_valid &= (tot<max_tot) & (tot > 0)

print(f'exclude {np.sum(~mask_valid)} events')

# the overflow is close to 2750 - it seems it is based on 2 bit and 2**38*1e-8
trb_overflow = 2**38*1e-8

# time in seconds, time_masked since epoch (1.1.1970); remove the overflow, 
# but don't do it at tot_time_ns as it can cause precision loss
# time_masked = np.unwrap(muon.file_handler.tot_time[mask_valid], period=trb_overflow)
time_masked = np.unwrap(tot_time_ns[mask_valid], period=trb_overflow)
time_masked += muon.file_handler.tot_time[0] - time_masked[0]

df_base = pandas.DataFrame({
    # time_masked since epoch (1.1.1970)
    'time': time_masked,
    # time_ns TRB internal
    'time_ns': tot_time_ns[mask_valid], 
    # channel id
    'channel': muon.file_handler.tot_channel[mask_valid], 
    # tot in nanoseconds
    'tot': tot[mask_valid]})

# # the time isn't sorted correctly, do it here
df_base.sort_values(['time', 'time_ns', 'tot'], inplace=True)

# free RAM - parameter not needed
del tot_time_ns, time_masked, mask_valid, tot

# add labels

# Plot

In [ ]:
# timeline of events

plt.figure()

# plot only the first n entries
n = 10000
sc = plt.scatter(strawb.tools.asdatetime(df_base.time.iloc[:n]),
            df_base.tot.iloc[:n],
            c=df_base.channel.iloc[:n])

plt.colorbar(sc, label='Channel')

ax = plt.gca()
ax.xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax.xaxis.get_major_locator())
)
ax.set_axisbelow(True)  # show grid behind scatter

plt.ylabel('ToT [ns]')
plt.xlabel('Date [s]')
plt.grid()
plt.tight_layout()

## Show short ToT's

In [ ]:
# gen. hist
counts, edges = np.histogram(df_base.tot, bins=1000)

# plot hist
plt.figure()
plt.stairs(counts, edges=edges)
plt.xlabel('ToT [ns]')
plt.ylabel('Counts')
plt.yscale('log')
plt.grid()
plt.tight_layout()

In [ ]:
# plot hist
plt.figure()

# gen. hist
dt = np.diff(df_base.time_ns)
counts, edges = np.histogram(dt[(dt>=0) &(dt<1e-1)], bins=1000)
plt.stairs(counts, edges=edges)

plt.yscale('log')
plt.xlabel('Delta time between events [s]')
plt.ylabel('Counts')
plt.grid()
plt.tight_layout()

# Event builder
Detect and build events. An event is defined as set of tot-events where the timestamps between the singel tot-events isn't greater as a limit (`dt_max`). Muons are traveling with ~c and 1ns corresponds to 0.3m distance. The MounTracker has a diameter of 13", so ~0.3m.

In [ ]:
# define the label function
def label_intermediate(input):
    """ Label features in an array based on a second intermediate array, which length is shorter by one.
    input                     =  [1,0,1,1,0,0,0,1]
    scipy.ndimage.label(input)=  [1,0,2,2,0,0,0,3]
    label_intermediate(input) = [1,1,2,2,2,0,0,3,3]

    PARAMETERS
    ----------
    input : array_like
        The intermediate array-like object to be labeled. Any non-zero values in `input` are
        counted as features and zero values are considered the background.

    RETURNS
    -------
    label : ndarray or int
        An integer ndarray where each unique feature in `input` has a unique
        label in the returned array. And each label is extended by one item.
    num_features : int
    Example
    -------
    >>> d_a = np.diff(a)
    >>> label = label_intermediate(d_a<1.)
    >>> assert len(label) == len(a)
    """
    label, num_features = scipy.ndimage.label(input, structure=None)
    label = np.append(label, [0])  # add one item
    label[1:][label[:-1]!=0] = label[:-1][label[:-1]!=0]  # add the shifted labels if label!=0
    return label, num_features

# test
a = np.array([1,0,2,2,0,0,0,3])
l, _ = label_intermediate(a)
print(f'input         :  {a}')
print(f'l_intermediate: {l}')

## Build events for dataset

In [ ]:
threshold_ns = 20e-9
# # add label to df
# df_base.sort_values(['time', 'time_ns'], inplace=True)

# evet selection
dt = np.diff(df_base['time_ns'])
# df_base['label'], _ = label_intermediate((dt>=0) & (dt<threshold_ns))
df_base['label'], _ = label_intermediate(np.abs(dt)<threshold_ns)

# remove all label 0, and make a copy of the dataframe, keep the base
df = df_base[df_base['label']>0].copy()
# df.reset_index(drop=True, inplace=True)

# add label count, but only if it doesn't exist
if 'label_count' not in df:
    df = pandas.merge(df, df.groupby('label')['label'].count(), how='left',
                      right_index=True, left_on='label', 
                      suffixes=(None, '_count'))
    
if 'time_ns_label' not in df:
    df = pandas.merge(df, df.groupby('label')['time_ns'].min(), how='left',
                      right_index=True, left_on='label', 
                      suffixes=(None, '_label'))
    df['time_ns_label'] = df.time_ns - df.time_ns_label

# # the time isn't sorted correctly, do it here
# df.sort_values(['time', 'time_ns_label', 'tot'], inplace=True)

print(f'events reduced from: {len(df_base)} to {len(df)} - or {len(df)/len(df_base)*100:.2f}%')
df

### Show one Event

In [ ]:
# get the events with most entries
event_mask = df.label_count==df.label_count.max()

# show the event
df[df.label==df.label[event_mask].iloc[0]]

# PLOT

In [ ]:
# plot the active channels per event
plt.figure()

label_counts = df.groupby('label')['label_count'].first()
plt.hist(label_counts, bins=np.arange(2, label_counts.max()+2)-.5)
plt.yscale('log')
plt.xlabel('Delta time between events [s]')
plt.ylabel('Counts')

plt.grid()
plt.tight_layout()

# Build events with multiplicity `length`

## check if one channel appears multiple times in one label
because we use a pivot table later

In [ ]:
# extract labels where one channel appears multiple times
mask_duplicate = df.duplicated(['label', 'channel'], keep=False)
df_duplicate = df[mask_duplicate].copy()

# cal. the time diff between the duplicate per label
if 'time_ns_duplicate' in df_duplicate:
    df_duplicate.pop('time_ns_duplicate')
df_duplicate = pandas.merge(df_duplicate, df_duplicate.groupby('label').time_ns.first(),
             how='left',
                      right_index=True, left_on='label', 
                      suffixes=(None, '_duplicate'))
df_duplicate['time_ns_duplicate'] = df_duplicate.time_ns - df_duplicate.time_ns_duplicate
df_duplicate

In [ ]:
# if time_ns_duplicate is not 0, there are two duplicates in on label
label_non_zero = df_duplicate[df_duplicate.time_ns_duplicate!=0].label.unique()

# show it
df_duplicate[df_duplicate.label.isin(label_non_zero)]

In [ ]:
# plot hist
plt.figure()

# gen. hist
counts_0, edges_0 = np.histogram(df.tot, bins=1000)

counts, edges = np.histogram(df.tot[mask_duplicate], bins=edges_0)
plt.stairs(counts, edges=edges,label='duplicates')

counts, edges = np.histogram(df.tot[~mask_duplicate], bins=edges_0)
plt.stairs(counts, edges=edges,label='non duplicates')

# plt.stairs(counts_0, edges=edges_0, label='all', color='gray', lw=1, alpha=1, ls=':')


plt.xlabel('ToT [ns]')
plt.ylabel('Counts')
plt.yscale('log')
plt.legend()
plt.grid()
# the plt.tight_layout()

## Result: 
- most of the events don't show duplicate entries per channel
- duplicate show a tot of ~25ns and ~58ns, whereas non-duplicate have ~25ns
- therefore -> the long TOT seem to be incorrect 
    - it is sorted by tot, and we want only the shortest
    - we get it with: aggfunc='first' 

In [ ]:
# get a dataframe with non duplicates
df_nondup = df[~mask_duplicate].copy()

print(f'events reduced from: {len(df)} to {len(df_nondup)} - or {len(df_nondup)/len(df)*100:.2f}%')

# add label count, but only if it doesn't exist
if 'label_count' not in df_nondup:
    df_nondup = pandas.merge(df_nondup, df_nondup.groupby('label')['label'].count(), how='left',
                      right_index=True, left_on='label', 
                      suffixes=(None, '_count'))
    
if 'time_ns_label' not in df:
    df_nondup = pandas.merge(df_nondup, df_nondup.groupby('label')['time_ns'].min(), how='left',
                      right_index=True, left_on='label', 
                      suffixes=(None, '_label'))
    df_nondup['time_ns_label'] = df_nondup.time_ns - df_nondup.time_ns_label
    
# add the channel of the first event to each label (to do the multiplicity plot)
if 'channel_first' not in df:
    df_nondup.sort_values('time_ns_label')
    df_nondup = pandas.merge(df_nondup, df_nondup.groupby('label')['channel'].first(), how='left',
                      right_index=True, left_on='label', 
                      suffixes=(None, '_first'))

# the time isn't sorted correctly, do it here
df_nondup.sort_values(['time', 'time_ns_label', 'tot'], inplace=True)

# multiplicity plot of hits per channel
Plot the 2d histogram of first channel hit to secondary channel hits for each label.

The x-axis is the channel of the first hit and the y-axis of the secondary channel hits. There should be no counts on the diagonal as duplicate channels are removed for each label.

In [ ]:
def get_nrows_ncols(number_subfigures, nrows=None, ncols=None):
    number_subfigures = int(number_subfigures)
    if (ncols is None and nrows is None) or \
       (ncols is not None and nrows is not None):
        raise ValueError(f'either ncols or nrows must be set. Both are set: ncols={ncols}, nrows={nrows} ')
    elif nrows is not None:
        nrows = int(nrows)
        ncols = int(np.ceil(number_subfigures/nrows))
    else:
        ncols = int(ncols)
        nrows = int(np.ceil(number_subfigures/ncols))
        
    active = np.zeros(int(nrows*ncols)).astype(bool)
    active[:number_subfigures] = True
    return nrows, ncols, active

get_nrows_ncols(3, ncols=3)

In [ ]:
nrows, ncols, ax_active = get_nrows_ncols(df_nondup.label_count.max(), ncols=3)
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, #layout="constrained", 
                       squeeze=False, sharex=True, 
                       figsize=(10,3*nrows),
#                        gridspec_kw=dict(height_ratios=[5,1])
                      )
ax_shape = ax.shape
ax = ax.flatten()

# remove the first channel entries
mask_events = df_nondup.channel != df_nondup.channel_first

for i in range(df_nondup.label_count.max()):
    if i==0:
        mask_label_count = df_nondup.label_count > 1
        title = 'all hits'
    else:
        mask_label_count = df_nondup.label_count == i+1
        title = f'hit multiplicity {i+1}'

    print(i, (mask_events&mask_label_count).sum())
        
    c, e_x, e_y = np.histogram2d(df_nondup.channel_first[mask_events&mask_label_count], 
                                 df_nondup.channel[mask_events&mask_label_count], 
                                 bins=np.arange(1, 18)-.5)
    c = np.ma.masked_equal(c, 0)
    
    cb = ax[i].pcolormesh(e_x, e_y, np.log10(c.filled(np.nan)), shading='auto')
    # cb = plt.matshow(np.log10(statistic_i.filled(np.nan)))

    fig.colorbar(cb, ax=ax[i], shrink=.6, label=f'log$_{{10}}$(counts)', location='right')
    ax[i].set_title(title)

for axi in ax[ax_active]:
    axi.axline((0,0), slope=1, ls='--', color='gray')
    axi.set_aspect('equal')
    axi.grid(lw=.5)
    axi.set_xlim(.5, 16.5)
    axi.set_xticks(ticks=np.arange(1,17))
    
    axi.set_ylim(16.5, .5)
    axi.set_yticks(ticks=np.arange(1,17))
    
ax = ax.reshape(ax_shape)
[axi.set_xlabel('channel first hit') for axi in ax[-1,:]]
[axi.set_ylabel('channel secondary hit') for axi in ax[:,0]]

for axi in ax.flatten()[~ax_active]:
    axi.set_axis_off()

plt.tight_layout()

# multiplicity plot of hits per scintillator

In [ ]:
df_nondup['scintillator'] = df_nondup.channel.copy()

# upper hemisphere is even, lower odd
for i, sipm_pair in enumerate([[1,11], [2,12], 
                               [3, 9], [4,10],
                               [5,15], [8,14], 
                               [7,13], [6,16]]):
    df_nondup.loc[df_nondup['channel'].isin(sipm_pair), 'scintillator'] = i+1
    
### add the channel of the first event to each label (to do the multiplicity plot)
if 'scintillator_first' in df_nondup:
    df_nondup.pop('scintillator_first')
df_nondup.sort_values('time_ns_label')
df_nondup = pandas.merge(df_nondup, df_nondup.groupby('label')['scintillator'].first(), how='left',
                  right_index=True, left_on='label', 
                  suffixes=(None, '_first'))
    
### add label count, but only if it doesn't exist
if 'scintillator_count' in df_nondup:
    df_nondup.pop('scintillator_count')
df_nondup = pandas.merge(df_nondup, df_nondup.groupby('label')['scintillator'].count(), how='left',
                  right_index=True, left_on='label', 
                  suffixes=(None, '_count'))
    

# the time isn't sorted correctly, do it here
df_nondup.sort_values(['time', 'time_ns_label', 'tot'], inplace=True)

### count how many (different) scintillator are involved at one label
if 'scintillator_double' in df_nondup:
    df_nondup.pop('scintillator_double')
df_nondup = pandas.merge(df_nondup, df_nondup.groupby(['label', 'scintillator']).scintillator.count(), 
                         how='left', 
                         right_index=True, left_on=['label','scintillator'], 
                         suffixes=(None, '_double'))
# this results in 1, if there is only one sipm involved or 2 if both sipms 
# for one scintillator are involved. True if both are involved:
df_nondup.scintillator_double = df_nondup.scintillator_double == 2 

if 'scintillator_double_count' in df_nondup:
    df_nondup.pop('scintillator_double_count')
# now count the scintillator_same_count for each label
df_nondup = pandas.merge(df_nondup, df_nondup.groupby('label')['scintillator_double'].sum(), how='left',
                  right_index=True, left_on='label', 
                  suffixes=(None, '_count'))
# if both sipms at one scintillator are involved it is 2
# if two scintillator are fully involved it is 4,...
# -> /= 2 to get the fully involved scintillator count
df_nondup.scintillator_double_count /= 2
df_nondup.scintillator_double_count = df_nondup.scintillator_double_count.astype(int)

# get 'scintillator_count' right
df_nondup['scintillator_count'] -= df_nondup['scintillator_double_count']

In [ ]:
df_nondup[df_nondup.scintillator_count == df_nondup.scintillator_count.max()]

In [ ]:
print('upper hemisphere is even, lower odd')

nrows, ncols, ax_active = get_nrows_ncols(df_nondup.scintillator_count.max(), ncols=3)
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, #layout="constrained", 
                       squeeze=False, sharex=True, 
                       figsize=(10,3*nrows),
#                        gridspec_kw=dict(height_ratios=[5,1])
                      )

ax_shape = ax.shape
ax = ax.flatten()

# remove the first channel entries
mask_events = df_nondup.scintillator != df_nondup.scintillator_first

for i in range(df_nondup.scintillator_count.max()):
    if i==0:
        mask_label_count = df_nondup.scintillator_count > 1
        title = 'all hits'
    else:
        mask_label_count = df_nondup.scintillator_count == i+1
        title = f'scintillator multiplicity {i+1}'
        
    c, e_x, e_y = np.histogram2d(df_nondup.scintillator_first[mask_events&mask_label_count], 
                                 df_nondup.scintillator[mask_events&mask_label_count], 
                                 bins=np.arange(1, 18)-.5)
    c = np.ma.masked_equal(c, 0)
    
    cb = ax[i].pcolormesh(e_x, e_y, np.log10(c.filled(np.nan)), shading='auto')
    # cb = plt.matshow(np.log10(statistic_i.filled(np.nan)))

    fig.colorbar(cb, ax=ax[i], shrink=.6, label=f'log$_{{10}}$(counts)', location='right')
    ax[i].set_title(title)


for axi in ax[ax_active]:
    axi.axline((0,0), slope=1, ls='--', color='gray')
    axi.set_aspect('equal')
    axi.grid(lw=.5)
    axi.set_xlim(.5, 8.5)
    axi.set_xticks(ticks=np.arange(1,9))
    
    axi.set_ylim(8.5, .5)
    axi.set_yticks(ticks=np.arange(1,9))
    
ax = ax.reshape(ax_shape)
[axi.set_xlabel('scintillator first hit') for axi in ax[-1,:]]
[axi.set_ylabel('scintillator secondary hit') for axi in ax[:,0]]


for axi in ax.flatten()[~ax_active]:
    axi.set_axis_off()

plt.tight_layout()

# multiplicity plot of full scintillator hits (both sipms saw something)

In [ ]:
print('upper hemisphere is even, lower odd')
nrows, ncols, ax_active = get_nrows_ncols(df_nondup.scintillator_double_count.max()+1, ncols=3)
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, #layout="constrained", 
                       squeeze=False, sharex=True, 
                       figsize=(10,3*nrows),
#                        gridspec_kw=dict(height_ratios=[5,1])
                      )

ax_shape = ax.shape
ax = ax.flatten()

# remove the first channel entries
mask_events = df_nondup.scintillator != df_nondup.scintillator_first

for i in range(df_nondup.scintillator_double_count.max()+1):
    if i==0:
        mask_label_count = df_nondup.scintillator_double_count > 0
        title = 'all hits'
    else:
        mask_label_count = df_nondup.scintillator_double_count == i
        title = f'scintillator multiplicity {i}'
        
    c, e_x, e_y = np.histogram2d(df_nondup.scintillator_first[mask_events&mask_label_count], 
                                 df_nondup.scintillator[mask_events&mask_label_count], 
                                 bins=np.arange(1, 18)-.5)
    c = np.ma.masked_equal(c, 0)
    
    cb = ax[i].pcolormesh(e_x, e_y, np.log10(c.filled(np.nan)), shading='auto')
    # cb = plt.matshow(np.log10(statistic_i.filled(np.nan)))

    fig.colorbar(cb, ax=ax[i], shrink=.6, label=f'log$_{{10}}$(counts)', location='right')
    ax[i].set_title(title)


for axi in ax[ax_active]:
    axi.axline((0,0), slope=1, ls='--', color='gray')
    axi.set_aspect('equal')
    axi.grid(lw=.5)
    axi.set_xlim(.5, 8.5)
    axi.set_xticks(ticks=np.arange(1,9))
    
    axi.set_ylim(8.5, .5)
    axi.set_yticks(ticks=np.arange(1,9))
    
ax = ax.reshape(ax_shape)
[axi.set_xlabel('scintillator first hit') for axi in ax[-1,:]]
[axi.set_ylabel('scintillator secondary hit') for axi in ax[:,0]]

for axi in ax.flatten()[~ax_active]:
    axi.set_axis_off()

# ax[-1].stairs(statistic_i.sum(axis=0), fill=True, edges=binned_stat.bin_edges[0])
# ax[-1].set_yscale('log')
plt.tight_layout()

# and now only the scintillator with double hits

In [ ]:
print('upper hemisphere is even, lower odd')
nrows, ncols, ax_active = get_nrows_ncols(df_nondup.scintillator_double_count.max()+1, ncols=3)
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, #layout="constrained", 
                       squeeze=False, sharex=True, 
                       figsize=(10,3*nrows),
#                        gridspec_kw=dict(height_ratios=[5,1])
                      )
ax_shape = ax.shape
ax = ax.flatten()

# remove the first channel entries
mask_events = df_nondup.scintillator != df_nondup.scintillator_first
mask_events &= df_nondup.scintillator_double

for i in range(df_nondup.scintillator_double_count.max()+1):
    if i==0:
        mask_label_count = df_nondup.scintillator_double_count > 0
        title = 'all hits'
    else:
        mask_label_count = df_nondup.scintillator_double_count == i
        title = f'scintillator multiplicity {i}'
        
    c, e_x, e_y = np.histogram2d(df_nondup.scintillator_first[mask_events&mask_label_count], 
                                 df_nondup.scintillator[mask_events&mask_label_count], 
                                 bins=np.arange(1, 18)-.5)
    c = np.ma.masked_equal(c, 0)
    
    cb = ax[i].pcolormesh(e_x, e_y, np.log10(c.filled(np.nan)), shading='auto')
    # cb = plt.matshow(np.log10(statistic_i.filled(np.nan)))

    fig.colorbar(cb, ax=ax[i], shrink=.6, label=f'log$_{{10}}$(counts)', location='right')
    ax[i].set_title(title)


for axi in ax[ax_active]:
    axi.axline((0,0), slope=1, ls='--', color='gray')
    axi.set_aspect('equal')
    axi.grid(lw=.5)
    axi.set_xlim(.5, 8.5)
    axi.set_xticks(ticks=np.arange(1,9))
    
    axi.set_ylim(8.5, .5)
    axi.set_yticks(ticks=np.arange(1,9))
    
ax = ax.reshape(ax_shape)
[axi.set_xlabel('scintillator first hit') for axi in ax[-1,:]]
[axi.set_ylabel('scintillator secondary hit') for axi in ax[:,0]]

for axi in ax.flatten()[~ax_active]:
    axi.set_axis_off()
    
# ax[-1].stairs(statistic_i.sum(axis=0), fill=True, edges=binned_stat.bin_edges[0])
# ax[-1].set_yscale('log')
plt.tight_layout()

# Extract the labels (sipm events) which have at least on full scintillator hit (both SiPMs saw something)

In [ ]:
df_double = df_nondup[df_nondup.scintillator_double_count > 0]

print(f'events reduced from: {len(df_nondup)} to {len(df_double)} - or {len(df_double)/len(df_nondup)*100:.2f}%')
df_double

In [ ]:
scintillator_unique = np.sort(df_double.scintillator.unique())

nrows, ncols, ax_active = get_nrows_ncols(len(scintillator_unique), ncols=3)
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, #layout="constrained", 
                       squeeze=False, sharex=True, 
                       figsize=(10,3*nrows),
#                        gridspec_kw=dict(height_ratios=[5,1])
                      )
ax_shape = ax.shape
ax = ax.flatten()

bins = np.arange(-df_double.time_ns_label.max(), df_double.time_ns_label.max(), .5e-9)
print(f'Bins: 0 to {df_double.time_ns_label.max()}, length: {len(bins)}')

list_fits = []
for i, s_i in enumerate(scintillator_unique):
    mask_scintillator = df_double.scintillator_double & (df_double.scintillator==s_i)
    ch_first = df_double[mask_scintillator].groupby(['label'])['channel'].first()
    t_0 = df_double[mask_scintillator].groupby(['label'])['time_ns'].min()
    t_1 = df_double[mask_scintillator].groupby(['label'])['time_ns'].max()
    dt = t_1 - t_0

    channels = np.sort(df_double[mask_scintillator].channel.unique())
        
    dt[ch_first==channels[0]] *= -1
    counts, edges = np.histogram(dt, bins=bins)
    ax[i].stairs(counts, edges=edges*1e9,label=f'dt: ch{channels[1]} - ch{channels[0]}',)
        
    ax[i].legend(loc='upper left', ncol=2, title=f'scintillator {s_i}')
    ax[i].grid()
    
ax = ax.reshape(ax_shape)
[axi.set_ylabel('counts') for axi in ax[:,0]]
[axi.set_xlabel('$\Delta$ t [ns]') for axi in ax[-1,:]]

for axi in ax.flatten()[~ax_active]:
#     axi.set_axis_off()
    axi.get_yaxis().set_visible(False)

plt.tight_layout()

In [ ]:
scintillator_unique = np.sort(df_double.scintillator.unique())

nrows, ncols, ax_active = get_nrows_ncols(len(scintillator_unique), ncols=3)
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, #layout="constrained", 
                       squeeze=False, sharex=True, 
                       figsize=(10,3*nrows),
#                        gridspec_kw=dict(height_ratios=[5,1])
                      )

ax_shape = ax.shape
ax = ax.flatten()

bins = np.arange(0, df_double.time_ns_label.max(), .2e-9)
print(f'Bins: 0 to {df_double.time_ns_label.max()}, length: {len(bins)}')

for i, s_i in enumerate(scintillator_unique):
    mask_scintillator = df_double.scintillator_double & (df_double.scintillator==s_i)
    mask_scintillator &= df_double.scintillator_double_count==2
    ch_first = df_double[mask_scintillator].groupby(['label'])['channel'].first()
    t_0 = df_double[mask_scintillator].groupby(['label'])['time_ns'].min()
    t_1 = df_double[mask_scintillator].groupby(['label'])['time_ns'].max()
    dt = t_1 - t_0

    channels = df_double[mask_scintillator].channel.unique()
    for ch_i in np.sort(channels):
        counts, edges = np.histogram(dt[ch_first==ch_i], bins=bins)
        ax[i].stairs(counts, edges=edges*1e9,label=f'{ch_i}')
        
    ax[i].legend(loc='upper right', ncol=2, title=f'scintillator {s_i} \nfirst event in channel:')
    ax[i].grid()
    
ax = ax.reshape(ax_shape)
[axi.set_ylabel('counts') for axi in ax[:,0]]
[axi.set_xlabel('$\Delta$ t [ns]') for axi in ax[-1,:]]

for axi in ax.flatten()[~ax_active]:
#     axi.set_axis_off()
    axi.get_yaxis().set_visible(False)

plt.tight_layout()

# Simulate gaussian offset between channel + gaussian signals

In [ ]:
bins = np.arange(-df_double.time_ns_label.max(), df_double.time_ns_label.max(), .5e-9)
print(f'Bins: 0 to {df_double.time_ns_label.max()}, length: {len(bins)}')

s_i = 2
mask_scintillator = df_double.scintillator_double & (df_double.scintillator==s_i)
ch_first = df_double[mask_scintillator].groupby(['label'])['channel'].first()
t_0 = df_double[mask_scintillator].groupby(['label'])['time_ns'].min()
t_1 = df_double[mask_scintillator].groupby(['label'])['time_ns'].max()
dt = t_1 - t_0

channels = np.sort(df_double[mask_scintillator].channel.unique())

dt[ch_first==channels[0]] *= -1
counts, edges = np.histogram(dt, bins=bins)
edges *= 1e9
x = strawb.tools.cal_middle(edges)

In [ ]:
import scipy.signal
import scipy.optimize

def gauss(x, *params):
    """Generalized sum of N gaussians. N is defined with: N=len(params)//3
    therefore len(params) must be a multiple of 3, i.e. 3, 6, 9,...
    and gaussian i is defined by: 
    x0_i = params[::3]
    sigma_i = params[1::3]
    scale_i = params[2::3]
    """
    y = np.zeros_like(x)
    for i in range(0, len(params), 3):
        y = y + params[i+2] * np.exp( -((x - params[i])/params[i+1])**2 / 2)
    return y

def fit_peaks_gauss(x, y, distance=None, height_min=None, height_max=None, verbose=2, min_peaks=None):
    if distance is None:
        distance = 1
    else:
        distance = int(distance / np.diff(x).mean())
        if distance<2:
            distance = 1
    if verbose<1:
        print('distance in index ', distance)
    i_pk, p_pk = scipy.signal.find_peaks(y, distance=distance,
                                             height=(height_min, height_max),
                                             threshold=(None, None),
                                             prominence=(None, None),
                                             width=(None, None),
#                                              plateau_size=(None, None)
                                        )

    # convert to dataframe; scipy.signal.find_peaks measures in indices, convert it here
    df_p = pandas.DataFrame(p_pk)
    for i in df_p.columns.intersection(
        ['left_edges', 'right_edges', 'left_bases', 'right_bases', 'left_ips', 'right_ips']):
        df_p[i] = np.interp(df_p[i], np.arange(len(x)), x)
    df_p['widths'] = df_p['right_ips'] - df_p['left_ips']
    df_p['peak_pos'] = x[i_pk]
    df_p[['x0', 'sigma', 'scale']] = np.nan 

    # fit single
    for i, row_i in df_p.iterrows():
        p0=[row_i.peak_pos, 
            row_i.widths/np.sqrt(2*np.log(2))/2,
            row_i.peak_heights]
        d_left = row_i.peak_pos - row_i.left_ips
        d_right = row_i.right_ips - row_i.peak_pos
        mask_data = (x>=row_i.peak_pos-+d_right*1.2) & (x<=row_i.peak_pos+d_right*1.2)
        if mask_data.sum()<=len(p0):
            ind = np.argwhere(x==row_i.peak_pos)[0]
            mask_data = (x>=x[ind-len(p0)]) & (x<=x[ind+len(p0)])
        try:
            # use relative error -> absolute_sigma=False, to concentrate on the peak
            popt, pcov = scipy.optimize.curve_fit(gauss, x[mask_data], y[mask_data], p0=p0)

            df_p.loc[i, 'x0'] = popt[0]
            df_p.loc[i, 'sigma'] = np.abs(popt[1])
            df_p.loc[i, 'scale'] = popt[2]
            if verbose<1:
                print(popt)
        except Exception as a:
            if verbose<2:
                print(i, a)

    # fit all together
    p0 = np.concatenate([df_p.peak_pos, 
                         df_p.widths/np.sqrt(2*np.log(2))/2,
                         df_p.peak_heights]).reshape(3,-1).T
    
    if min_peaks is not None and len(p0) <= min_peaks:
        p0 = np.append(p0, [0,1,1]*(min_peaks-len(p0)))

    # use absolute error -> absolute_sigma=True+sigma, to NOT concentrate on the peaks
    popt, pcov = scipy.optimize.curve_fit(gauss, x, y, 
                                          p0=p0.flatten(),
                                          absolute_sigma=True, 
                                          sigma=np.sqrt(np.abs(counts))+1,
                                         )

    # merg the two dataframes
    df_full = pandas.DataFrame(columns=['x0', 'sigma', 'scale'], 
                     data= popt.reshape(-1,3))
    df_full.sort_values('x0', inplace=True, ignore_index=True)
    df_p = df_p.merge(df_full, how='outer', left_index=True, right_index=True, suffixes=['_single', None])

    return df_p

In [ ]:
df_p = fit_peaks_gauss(x, counts)

In [ ]:
scintillator_unique = np.sort(df_double.scintillator.unique())

nrows, ncols, ax_active = get_nrows_ncols(len(scintillator_unique), ncols=3)
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, #layout="constrained", 
                       squeeze=False, sharex=True, 
                       figsize=(10,3*nrows),
#                        gridspec_kw=dict(height_ratios=[5,1])
                      )
ax_shape = ax.shape
ax = ax.flatten()

bins = np.arange(-df_double.time_ns_label.max(), df_double.time_ns_label.max(), .5e-9)
print(f'Bins: 0 to {df_double.time_ns_label.max()}, length: {len(bins)}')

list_fits = []
for i, s_i in enumerate(scintillator_unique):
    mask_scintillator = df_double.scintillator_double & (df_double.scintillator==s_i)
    ch_first = df_double[mask_scintillator].groupby(['label'])['channel'].first()
    t_0 = df_double[mask_scintillator].groupby(['label'])['time_ns'].min()
    t_1 = df_double[mask_scintillator].groupby(['label'])['time_ns'].max()
    dt = t_1 - t_0

    channels = np.sort(df_double[mask_scintillator].channel.unique())
        
    dt[ch_first==channels[0]] *= -1
    counts, edges = np.histogram(dt, bins=bins)
    ax[i].stairs(counts, edges=edges*1e9,label=f'dt: ch{channels[1]} - ch{channels[0]}',)
    
    # fit peaks
    x_fit = strawb.tools.cal_middle(edges)*1e9
    df_p = fit_peaks_gauss(x_fit, counts, distance=3, height_min=counts.max()/10, min_peaks=2)
    df_p['scintillator'] = s_i
    list_fits.append(df_p)
    x_plot = np.linspace(x_fit.min(), x_fit.max(), 1000)
    popt = df_p[['x0', 'sigma', 'scale']]
    lin, = ax[i].plot(x_plot, gauss(x_plot, *popt.to_numpy().flatten()), '-', label='fit')
    for j, row_j in df_p[['x0', 'sigma', 'scale']].iterrows():
        ax[i].plot(x_plot, gauss(x_plot, row_j.x0, row_j.sigma, row_j.scale), '--', zorder=0,
#                    color=lin.get_color()
                  )

#     popt = df_p[['x0_single', 'sigma_single', 'scale_single']]
#     popt = popt[~popt.isna().any(axis=1)]  # exclude missing values
#     lin, = ax[i].plot(x_plot, gauss(x_plot, *popt.to_numpy().flatten()), '--', label='fit single')
#     for i, row_i in popt.iterrows():
#         ax[i].plot(x_plot, gauss(x_plot, row_i.x0_single, row_i.sigma_single, row_i.scale_single), ':', color=lin.get_color())

        
    ax[i].legend(loc='upper left', ncol=2, title=f'scintillator {s_i}')
    ax[i].grid()
    
ax = ax.reshape(ax_shape)
[axi.set_ylabel('counts') for axi in ax[:,0]]
[axi.set_xlabel('$\Delta$ t [ns]') for axi in ax[-1,:]]

for axi in ax.flatten()[~ax_active]:
#     axi.set_axis_off()
    axi.get_yaxis().set_visible(False)

plt.tight_layout()

# combine all fit df's and take only the columns of interest
columns = ['scintillator', 
           'x0', 'sigma', 'scale', 
           'x0_single', 'sigma_single', 'scale_single', 
           'peak_heights', 'peak_pos']
df_fits = pandas.concat(list_fits, ignore_index=True)[columns]

df_fits

# Simulate the signal

In [ ]:
def sper2cart(radius, phi, theta=None):
    """Converts sperical or polar coordinates to cartesian.
    sph2cart(1, 0) -> x,y = (1 , 0) 
    sph2cart(1, 0, 0) -> x,y,z = (0, 0, 1)
    PARAMETER
    ---------
    radius: int, float, list or ndarray
    phi: int, float, list or ndarray
        should be in the range from [0, 2*np.pi]
    theta: int, float, list or ndarray, optional
        set theta (is not None) to use sperical coordinates
        should be in the range from [0, np.pi]
    RETURN
    ------
    x: float or ndarray
    y: float or ndarray
    z: float or ndarray, optional
        if theta is not None
    """
    rr, pp, tt = np.broadcast_arrays(radius, phi, theta)    
    if theta is None:
        x = np.cos(pp) * rr
        y = np.sin(pp) * rr
        return x, y
    else:
        x = np.sin(tt) * np.cos(pp) * rr
        y = np.sin(tt) * np.sin(pp) * rr
        z = np.cos(tt) * rr
        return x, y, z
    
    
def cart2sper(x, y, z=None):
    """Converts cartesian to sperical (z!=None) or polar coordinates.
    cart2sph(1, 0) -> r,phi = (1 , 0) 
    cart2sph(0, 0, 1) -> r,phi,theta = (1, 0, 0)
    PARAMETER
    ---------
    x: int, float, list or ndarray
    y: int, float, list or ndarray
    z: int, float, list or ndarray, optional
        set z (is not None) to use sperical coordinates
    RETURN
    ------
    radius: float or ndarray
    phi: float or ndarray
        in the range from [0, 2*np.pi]
    theta: float or ndarray, optional
        in the range from [0, np.pi]
    """
    xx, yy, zz = np.broadcast_arrays(x, y, z)    
    
    r_xy = np.hypot(xx, yy)
    phi = np.arctan2(yy, xx) % (np.pi * 2.)
    if z is None:
        return r_xy, phi
    
    r_xyz = np.hypot(zz, r_xy)
    theta = np.pi/2 - np.arctan2(zz, r_xy) 
    return r_xyz, phi, theta

def min_max(x, axis=-1):
    return np.array([np.min(x, axis=axis), np.max(x, axis=axis)])

In [ ]:
pos_0 = np.array([.070/2, .070/2, .01])
pos_1 = np.array([-.070/2, -.070/2, -.01])

n = int(1e4)

# sphere
# theta = np.random.random_sample(n) * np.pi
# phi = np.random.random_sample(n) * np.pi*2.

# pos_mu = np.array(sper2cart(radius=.075, phi=phi, theta=theta))

pos_mu = np.array([(np.random.random_sample(n)-.5) * .075,
                   (np.random.random_sample(n)-.5) * .075,
                   (np.random.random_sample(n)-.5) * .02])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(*pos_mu, s=3, alpha=.1)
ax.scatter(*pos_0)
ax.scatter(*pos_1)

In [ ]:
pos_0 = np.array([.075/2, .075/2, .01])
pos_1 = np.array([-.075/2, -.075/2, -.01])

n = int(1e6)

# use different start points
# sphere
pos_mu = np.array(sper2cart(radius=.075,
                            phi=np.random.random_sample(n) * np.pi*2., 
                            theta=np.random.random_sample(n) * np.pi))

# plane
# pos_mu = np.array([(np.random.random_sample(n)-.5) * .075,
#                    (np.random.random_sample(n)-.5) * .075,
#                     np.ones(n) * 0.01])

# rectangle
# pos_mu = np.array([(np.random.random_sample(n)-.5) * .075,
#                    (np.random.random_sample(n)-.5) * .075,
#                    (np.random.random_sample(n)-.5) * .0001])

# calculate the times of the SiPM
t_s1 = cart2sper(*(pos_mu - pos_0.reshape(-1,1)))[0] / 2.99e8 *  1.5 - .05e-9
t_s1 += np.random.normal(size=len(t_s1), scale=.005e-9)
t_s2 = cart2sper(*(pos_mu - pos_1.reshape(-1,1)))[0] / 2.99e8 *  1.5
t_s2 += np.random.normal(size=len(t_s1), scale=.007e-9)

bins_s = np.arange(-10, 10, .001) *  1e-10

# t_0 = np.concatenate([t_s1, t_s2])
t_0 = t_s1 - t_s2
# t_0 += np.random.normal(loc=5, scale=1, size=len(t_0))

counts, edges = np.histogram(t_0, bins=bins_s*1e9)
x = strawb.tools.cal_middle(edges)

fig, ax = plt.subplots(ncols=1, nrows=1, #layout="constrained", 
                       squeeze=False, sharex=False, 
                       figsize=(6,4),
#                        gridspec_kw=dict(height_ratios=[5,1])
                      )
ax = ax.flatten()

counts, edges = np.histogram(t_0, bins=bins_s)
ax[0].stairs(counts, edges=edges)

counts, edges = np.histogram(t_s1, bins=bins_s)
ax[0].stairs(counts, edges=edges)
counts, edges = np.histogram(t_s2, bins=bins_s)
ax[0].stairs(counts, edges=edges)